## Kelley hu4d5-5 VL-R66G vdw step error analysis

In this notebook we will use a random forest model to find the most energetically influential degrees of freedom for the VL-R66G vdw step TI production run. Next, we will compare the sampling of these DOF during TI production to a free energy profile derived from end state GaMD sampling. We will attempt to  correct any inaccurate sampling in the TI data and find the estimated ddG before and after the correction. 

In [1]:
import os
os.chdir("..")
from common_functions import *

### Ingesting original TI lambda production data

In [2]:
os.chdir("./TI_data/VL-R66G")
geom_dvdls_crg = pd.read_csv("R66G_crg_bound.csv")
geom_dvdls_crg_ub = pd.read_csv("R66G_crg_unbound.csv")
geom_dvdls_vdw = pd.read_csv("R66G_vdw_bound.csv")
geom_dvdls_vdw_ub = pd.read_csv("R66G_vdw_unbound.csv")


### Initial ddG estimate:

In [3]:
dG_bd_crg = geom_dvdls_crg.groupby("Lambda").mean()["weight_dvdl"].sum()
dG_ubd_crg = geom_dvdls_crg_ub.groupby("Lambda").mean()["weight_dvdl"].sum()
ddG_crg = dG_bd_crg - dG_ubd_crg

dG_bd_vdw = geom_dvdls_vdw.groupby("Lambda").mean()["weight_dvdl"].sum()
dG_ubd_vdw = geom_dvdls_vdw_ub.groupby("Lambda").mean()["weight_dvdl"].sum()
ddG_vdw = dG_bd_vdw - dG_ubd_vdw

empirical_value = 0.22

print("Original ddG (crg step): ")
print(f"{round(ddG_crg, 4)} kcal/mol")

print("Original ddG (vdw step): ")
print(f"{round(ddG_vdw, 4)} kcal/mol")

print("Original total ddG: ")
print(f"{round(ddG_crg + ddG_vdw, 4)} kcal/mol")

print()
print("Empirical value: ")
print(f"{empirical_value} kcal/mol")

orig_error = abs((ddG_crg + ddG_vdw) - empirical_value)

print("Original ddG error: ")
print(f"{round(orig_error, 4)} kcal/mol")

Original ddG (crg step): 
-0.8642 kcal/mol
Original ddG (vdw step): 
0.1575 kcal/mol
Original total ddG: 
-0.7066 kcal/mol

Empirical value: 
0.22 kcal/mol
Original ddG error: 
0.9266 kcal/mol


### Vdw step RF model

#### Splitting data into independent/dependent variables for random forest model

See our methods/supplemental methods section for our process to choose the input features.

In [4]:
X = geom_dvdls_vdw.drop(
    ["weight_dvdl", "dvdl", "Run", "Lambda", "#Frame",
    ], axis=1)

X_scl = pd.DataFrame(StandardScaler().fit_transform(X))
X_scl.columns = X.columns
Y = geom_dvdls_vdw["weight_dvdl"]


#### Checking to see if there is any cross-correlations within the dataset

In [5]:
absCorr = abs(X_scl.corr())
for i in absCorr.columns:
    for j in absCorr.index:
        cor = absCorr.loc[i, j]
        if abs(cor) > 0.5 and i != j:
            print(i, j)
            print(cor)
            

#### Using random forest model to identify the most energetically influential degrees of freedom

We run our model 25 times, then sort the results by the mean of feature importance across the 25 iterations. For this particular perturbation, the model $R^2$ between the geometric DOF (nearby side chain rotamers or interatomic distances) and the energetic DV/DL was not strong enough for us to check the sampling of these degrees of freedom. 

In [9]:
rfeDefault = RFE(estimator=DecisionTreeRegressor(max_depth=5, random_state=42), n_features_to_select=0.75, step=0.05)
rfDefault = RandomForestRegressor(
    max_depth=10, n_estimators=200, oob_score=True, max_features=0.6, min_samples_leaf = 7, min_samples_split=14, random_state=42
)

pipelineDefault_rf = Pipeline([
    ('feature_scaling', StandardScaler()),
    ('feature_selection', rfeDefault),
    ('regression_model', rfDefault)
])


imps = benchmark_model(pipelineDefault_rf, X_scl, Y, geom_dvdls_vdw["Lambda"])
imps[["Mean", "Median"]].sort_values(by="Mean", ascending=False)[:15]

Avg. training r2: 
0.5685
Training r2 std dev: 
0.0015
Avg. test r2: 
0.4325
Testing r2 std dev: 
0.0063


,Mean,Median
F426_chi2,0.165513,0.165693
N385_chi1,0.154993,0.154210
S407_chi1,0.141395,0.143542
S420_chi1,0.131758,0.142609
T386_chi1,0.115618,0.110394
T427_chi1,0.114754,0.112565
N385_chi2,0.101531,0.098184
S422_chi1,0.074438,0.075353
